해당 코드는 로컬 PC에서 작성 및 실행한 코드입니다.   
각 팀원이 동시에 여러 모델을 학습시키는 과정에서   
YOLO, Mobilenet, Inceoption 모델은 로컬 PC를 사용하게 되었습니다.   
   
사용한 파이썬 버전과 라이브러리 목록입니다.   
python : 3.12.3   
torch : 2.3.0   
ultralytics : 8.2.22   

In [ ]:
import os

# label 만들 이미지 폴더 경로
image_folder_path = '../preprocessed_crop/img/train/anger'

# label 저장 폴더 경로
label_folder_path = '../preprocessed_crop/label/train/anger'

# 폴더 생성
os.makedirs(label_folder_path, exist_ok=True)

# 이미지 파일 확장자
image_extensions = ['.jpg', '.jpeg']

# 이미지 폴더 내 모든 파일 작업
for filename in os.listdir(image_folder_path):
    if any(filename.lower().endswith(ext) for ext in image_extensions):
        base_name = os.path.splitext(filename)[0] # 확장자 제거
        txt_file_path = os.path.join(label_folder_path, base_name + '.txt')
        with open(txt_file_path, 'w') as f:
            f.write('0 0.5 0.5 1 1')
            """
            데이터셋 형식
            label (0: anger, 1: happy, 2: panic, 3: sadness)
            box의 x좌표 중심
            box의 y좌표 중심
            box의 넓이
            box의 높이
            """
print("라벨 파일 생성 완료")

In [ ]:
# YOLO 모델 학습을 위한 yaml 파일 생성

import yaml

data = {'train' : '../yolov8/train/', # train 데이터셋 경로
        'val' : '../yolov8/val/', # val 데이터셋 경로
        'name' : {0:'anger', 1:'happy', 2:'panic', 3:'sadness'}, # 클래스별 이름
        'nc' : 4 } # 클래스 수

with open('../yolov8/Data.yaml', 'w') as f:
  yaml.dump(data, f)

with open('../yolov8/Data.yaml', 'r') as f:
  data_yaml = yaml.safe_load(f)
  display(data_yaml)


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.yaml') #YOLOV8n 모델 활용

training_params = {
    'data': 'C:/Users/handt/Desktop/finalpjt data/yolov8/Data.yaml',
    'epochs': 10,
    'batch': 32,
    'imgsz': 416,
    'optimizer': 'auto',
    'lrf': 0.01,
    'weight_decay': 0.0005,
    'augment': True,
}
model.train(**training_params) #모델 학습

In [ ]:
from ultralytics import YOLO

model = YOLO('best.pt') #학습된 모델 불러오기
results = model('../yolov8/test/13.jpg', save_txt=True) #예측 결과 확인


In [ ]:
# 테스트 데이터 예측 정확도 확인을 위해, 폴더 내 모든 이미지 파일을 예측하여 저장하는 코드

import os
from pathlib import Path
from ultralytics import YOLO

# YOLO 모델 불러오기
model = YOLO('best.pt')

# 이미지 폴더 경로
image_folder = Path('C:/Users/handt/Desktop/finalpjt data/TEST_DATA_SET/test_crop/4. sadness')

# 처리할 이미지의 확장자 목록
extensions = ['*.jpg', '*.jpeg']

# 폴더 내 모든 이미지 파일 예측
for ext in extensions:
    for image_path in image_folder.glob(ext):
        results = model(str(image_path), save_txt=True)

print("예측 결과 저장 완료")
